## Plot markers umap embeddings
Does this over each donor and separate

In [1]:
# Input info
allSamples_f <- "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/allSamples.rds"
#se_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/SE.rds"
sample_names = "preB,postB"
outdir = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome"
#"/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/umap_markers"

# Parameters
assay="RNA"

# Parallel processing
cores = 4

# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"

gene_file = "/data/Mito_Trace/data/processed/genomes/mtMasked/GRCh38_MT_blacklist/genes/genes.gtf"

In [2]:
source(utils_script)
dir.create(outdir, showWarnings = FALSE)


Attaching SeuratObject



In [3]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
sample_names

[1] "preB"  "postB"

In [4]:
allSamples <- readRDS(allSamples_f)
allSamples

[[1]]
An object of class Seurat 
40000 features across 1351 samples within 1 assay 
Active assay: ATAC (40000 features, 0 variable features)

[[2]]
An object of class Seurat 
41874 features across 2890 samples within 1 assay 
Active assay: ATAC (41874 features, 0 variable features)


In [5]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")

library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

library(dplyr)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

Lo

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."

Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:ensembldb':

    filter, select


The following object is masked from 'package:AnnotationDbi':

    select


The following object is masked from 'package:Biobase':

    combine


The following objects are masked from 'package:GenomicRanges':

    intersect, setdiff, union


The following object is masked from 'package:GenomeInfoDb':

    intersect


The following objects are masked from 'package:IRanges':

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from 'package:S4Vectors':

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from 'package

In [6]:
# annotations <- makeGRangesFromGTF(
#       gene_file,
#       level = c("genes", "transcripts"),
#       ignoreTxVersion = TRUE,
#       .checkAgainstTxDb = FALSE
#     )

In [7]:
# extract gene annotations from EnsDb
#annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v75)
gffRangedData<-rtracklayer::import.gff(gene_file, format="gtf", genome="hg38")
annotations<-as(gffRangedData, "GRanges")
values(annotations)$gene_biotype <- values(annotations)$gene_type
annotations


Attaching package: 'Biostrings'


The following object is masked from 'package:base':

    strsplit


Warning message in readGFFAsGRanges(resource, version = version, colnames = colnames, :
"cannot set the seqlengths or circularity flags on the GRanges object to
  return because the sequence names in the GTF or GFF file are in
  disagreement with the sequence names implied by the genome assembly
  (hg38) specified via the 'genome' argument"


GRanges object with 2765969 ranges and 25 metadata columns:
              seqnames        ranges strand |   source       type     score
                 <Rle>     <IRanges>  <Rle> | <factor>   <factor> <numeric>
        [1]       chr1   29554-31109      + |   HAVANA gene              NA
        [2]       chr1   29554-31097      + |   HAVANA transcript        NA
        [3]       chr1   29554-30039      + |   HAVANA exon              NA
        [4]       chr1   30564-30667      + |   HAVANA exon              NA
        [5]       chr1   30976-31097      + |   HAVANA exon              NA
        ...        ...           ...    ... .      ...        ...       ...
  [2765965] KI270734.1 138483-138667      - |  ENSEMBL CDS               NA
  [2765966] KI270734.1 138480-138482      - |  ENSEMBL stop_codon        NA
  [2765967] KI270734.1 161689-161852      - |  ENSEMBL UTR               NA
  [2765968] KI270734.1 161587-161626      - |  ENSEMBL UTR               NA
  [2765969] KI270734.1 13808

In [8]:
# ensdb.annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v75)
# ensdb.annotations

# seqlevelsStyle(ensdb.annotations) <- 'UCSC'
# ensdb.annotations

# sort(ensdb.annotations[(elementMetadata(ensdb.annotations)[, "gene_name"] %in% c("LYZ"))])


# sort(annotations[(elementMetadata(annotations)[, "gene_name"] %in% c("LYZ"))])


In [9]:
markers_f <- "/data/Mito_Trace/data/processed/pbmc_markers.txt"
markers <- read.csv(markers_f, header=F)[[1]]

#small.umap.outdir <- file.path(outdir, "markersDimRed")
#dir.create(small.umap.outdir)


In [10]:
curr.se <- allSamples[[1]]
# add the gene information to the object
Annotation(curr.se) <- annotations
gene.activities <- GeneActivity(curr.se)


Extracting gene coordinates

Extracting reads overlapping genomic regions



In [16]:
source(utils_script)
#curr.sample <- curr.sample

run.sample.sep <- function(curr.sample, outdir){
    print("outdir")
    print(outdir)
    DefaultAssay(curr.sample) <- "ATAC"

    Annotation(curr.sample) <- annotations
    gene.activities <- GeneActivity(curr.sample)

    curr.outdir <- file.path(outdir, curr.sample$orig.ident[[1]])
    dir.create(curr.outdir)
    out <- embed.atac(curr.sample)
    curr.sample <- out[[1]]
    # add the gene activity matrix to the Seurat object as a new assay and normalize it
    curr.sample[['RNA']] <- CreateAssayObject(counts = gene.activities)
    curr.sample <- NormalizeData(
      object = curr.sample,
      assay = 'RNA',
      normalization.method = 'LogNormalize',
      scale.factor = median(curr.sample$nCount_RNA)
    )

    saveRDS(curr.sample, file.path(curr.outdir, paste0("se.rds")))
    DimPlot(curr.sample)
    ggsave(file.path(curr.outdir, "dimPlot.png"))
    
    feature.names <- ClosestFeature(curr.sample, rownames(curr.sample))
    marker.feature.names <- feature.names %>% filter(gene_name %in% markers)
    rownames(marker.feature.names) <- marker.feature.names[,"query_region"]
    # marker.feature.names
    marker.feature.names$name <- paste0(marker.feature.names$gene_name, "_", marker.feature.names$type)
    sapply(rownames(marker.feature.names), 
            FUN=featplot, se=curr.sample, curr.outdir=curr.outdir, feat.names=marker.feature.names["name"])


    DefaultAssay(curr.sample) <- "RNA"
    sapply(markers, 
        FUN=featplot, se=curr.sample, curr.outdir=curr.outdir, feat.names=NULL)
    
}

sapply(allSamples, FUN=run.sample.sep, outdir=outdir)

[1] "outdir"
[1] "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome"


Extracting gene coordinates

Extracting reads overlapping genomic regions

Warning message in dir.create(curr.outdir):
"'/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/preB' already exists"
Performing TF-IDF normalization

Running SVD

Scaling cell embeddings

Warning message:
"The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session"
14:48:47 UMAP embedding parameters a = 0.9922 b = 1.112

14:48:47 Read 1351 rows and found 49 numeric columns

14:48:47 Using Annoy for neighbor search, n_neighbors = 30

14:48:47 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*


[1] "Feature no in object: TCRA"
[1] "Feature no in object: CD56"
[1] "Feature no in object: CD161"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "Feature no in object: IGHM"
[1] "Feature no in object: TCRA"


...

[1] "Feature no in object: TCRA"
[1] "Feature no in object: CD56"
[1] "Feature no in object: CD161"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "Feature no in object: IGHM"
[1] "Feature no in object: TCRA"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



TCRA,Feature no in object: TCRA,Feature no in object: TCRA
CD56,Feature no in object: CD56,Feature no in object: CD56
CD161,Feature no in object: CD161,Feature no in object: CD161
NCR1,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/preB/NCR1.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/postB/NCR1.embedFeat.top.png
NCAM1,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/preB/NCAM1.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/postB/NCAM1.embedFeat.top.png
CEACAM8,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/preB/CEACAM8.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/postB/CEACAM8.embedFeat.top.png
IGHM,Feature no in object: IGHM,Feature no in object: IGHM
TCRA,Feature no in object: TCRA,Feature no in object: TCRA
IL7R,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/preB/IL7R.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/postB/IL7R.embedFeat.top.png
CCR7,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/preB/CCR7.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/postB/CCR7.embedFeat.top.png
CD14,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/preB/CD14.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds_customgenome/postB/CD14.embedFeat.top.png


In [ ]:
sessionInfo()